In [ ]:
import random
import numpy as np

from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit, transpile, Aer, execute
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Shor
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_bloch_vector, plot_histogram

## Qubit

In [ ]:
#plot_bloch_vector([1,0,0], coord_type = "spherical")
#plot_bloch_vector([1,np.pi,0], coord_type = "spherical")
#plot_bloch_vector([1,np.pi/2,0], coord_type = "spherical")
plot_bloch_vector([1,np.pi/2,np.pi/2], coord_type = "spherical")
#plot_bloch_vector([1,np.pi/2,np.pi], coord_type = "spherical")
#plot_bloch_vector([1,1.1*np.pi/3,np.pi/6], coord_type = "spherical")

## Quantum Gates

In [ ]:
state = Statevector.from_int(0, 2)
state.draw('latex')

In [ ]:
circ = QuantumCircuit(1,1)
circ.h(0)
circ.draw('mpl')

In [ ]:
state = state.evolve(circ)
state.draw('latex')

In [ ]:
state = Statevector.from_int(0, 2)
circ = QuantumCircuit(1,1)
circ.u(np.pi/2,0,np.pi,0)
circ.draw('mpl')

In [ ]:
state = state.evolve(circ)
state.draw('latex')

## Quantum Circuits

In [ ]:
state = Statevector.from_int(0, 4)
circ = QuantumCircuit(2,2)
circ.h(0)
circ.x(1)
circ.draw('mpl')

In [ ]:
state = state.evolve(circ)
state.draw('latex')

## Superposition

In [ ]:
state = Statevector.from_int(0, 2)
circ = QuantumCircuit(1,1)
circ.h(0)
circ.draw('mpl')

In [ ]:
backend = Aer.get_backend('statevector_simulator')
job = backend.run(circ)
result = job.result()
outputstate = result.get_statevector(circ, decimals=3)
print(outputstate)

In [ ]:
state = Statevector.from_int(0, 2)
circ = QuantumCircuit(1,1)
circ.h(0)
circ.measure(range(1), range(1))
circ.draw('mpl')

In [ ]:
backend = Aer.get_backend('qasm_simulator')
qc_compiled = transpile(circ, backend)
job_sim = backend.run(qc_compiled, shots=1)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_compiled)
print(counts)

In [ ]:
plot_histogram(counts)

## Entanglement

In [ ]:
meas = QuantumCircuit(2, 2)
meas.h(0)
meas.cx(0, 1)
meas.measure([0,1], [0,1])
meas.draw('mpl')

In [ ]:
backend = Aer.get_backend('qasm_simulator')
qc_compiled = transpile(meas, backend)
job_sim = backend.run(qc_compiled, shots=1000)
result_sim = job_sim.result()
counts = result_sim.get_counts(qc_compiled)
print(counts)

In [ ]:
plot_histogram(counts)

## Teleportation

In [ ]:
a1 = random.random()*2 -1 
a2 = random.random()*2 -1
b1 = random.random()*2 -1
b2 = random.random()*2 -1
norm = (a1**2 + a2**2 + b1**2 + b2**2)**0.5
c1 = complex(a1/norm,a2/norm) #Amplitude for |0> 
c2 = complex(b1/norm,b2/norm) #Amplitude for |1>

print("Alice's qubit is:")
print("{} |0> + {} |1>".format(c1, c2))

In [ ]:
psi  = QuantumRegister(1, name = 'psi')   # The qubit to teleport
bell = QuantumRegister(2, name = 'bell')  # The shared entangled pair 
c    = ClassicalRegister(2, name = 'c')   # Two classical bits for the measures
teleport = QuantumCircuit(psi,bell,c)     # We create the circuit with the two quantum registers and the classical bits
teleport.initialize([c1,c2],psi)          # We set the amplitudes for Alice's quibt
teleport.barrier()
teleport.h(bell[0])
teleport.cx(bell[0],bell[1])
teleport.barrier()
teleport.cx(psi,bell[0])
teleport.h(psi)
teleport.measure([psi[0],bell[0]],c)
teleport.barrier()
teleport.cx(bell[0],bell[1])
teleport.cz(psi,bell[1])
teleport.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('statevector_simulator')
job = execute(teleport, backend)
outputstate = job.result().get_statevector()
outputstate = np.array([outputstate]).T
print("Bob's qubit is:")
print("{} |0> + {} |1>".format(outputstate[np.nonzero(outputstate)[0][0]], outputstate[np.nonzero(outputstate)[0][1]]))

## Deutsch-Jozsa algorithm

In [ ]:
n = 4                               # Number of qubits that we are going to use in the oracle
q   = QuantumRegister(n, 'q')       # The oracle qubits
out = QuantumRegister(1, 'out')     # Qubit for the oracle output
c   = ClassicalRegister(n, 'c')     # Classical bits needed for the result of the measurement
circ_init = QuantumCircuit(q,out,c) # Initial part of the circuit

for i in range(n):    
    circ_init.h(q[i]) # We apply H to all the oracle qubits
    
circ_init.x(out)    # We apply X and H to the output qubit
circ_init.h(out)
circ_init.barrier() # Visual barrier to separate the parts of the circuit

circ_end = QuantumCircuit(q,out,c)
circ_end.barrier()   # Visual barrier to separate the parts of the circuit
for i in range(n):    
    circ_end.h(q[i]) # We apply H to all the oracle qubits    
circ_end.measure(q,c)

const = QuantumCircuit(q,out,c)
const.cx(q[0],out)
const.x(q[0])
const.cx(q[0],out)
const.x(q[0])

bal = QuantumCircuit(q,out,c)
bal.cx(q[0],out)

In [ ]:
circ_const = circ_init + const + circ_end
circ_const.draw(output='mpl')

In [ ]:
backend = Aer.get_backend('qasm_simulator')
job = execute(circ_const, backend, shots=1)
counts = job.result().get_counts()
print(counts)

In [ ]:
circ_bal = circ_init + bal + circ_end
circ_bal.draw(output='mpl')

In [ ]:
job = execute(circ_bal, backend, shots=1)
counts = job.result().get_counts()
print(counts)

## Shor

In [ ]:
backend = Aer.get_backend('qasm_simulator')
quantum_instance = QuantumInstance(backend)
shor = Shor(quantum_instance=quantum_instance)
result = shor.factor(N=35, a=2)
print('Factors:', result.factors)